In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Load TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs/scalars

In [ ]:
gesture_map = {'none':0, 'click':1, 'doubleclick':2, 'rightcircle':3, 'leftcircle':4}
path = '../data/session2/'
users = ['david']
gestures = ['none', 'click', 'doubleclick', 'rightcircle', 'leftcircle']
num_takes = 10
num_classes = len(gestures)

In [ ]:
# Find minimum common sequence length
min_count = 1000
for u in users:
    for g in gestures:
        for n in range(1, num_takes+1):
            fname = path + u + '_' + g + '_' + str(n) + '.csv'
            with open(fname, 'r') as f:
                header = f.readline()
                count = 0
                for data_point in f:
                    count += 1
                if count < min_count:
                    min_count = count
seq_length = min_count

In [ ]:
# Load in data
data_dict = {}
for u in users:
    user_batch = []
    user_labels = []
    for g in gestures:
        for nt in range(1, num_takes+1):
            fname = path + u + '_' + g + '_' + str(nt) + '.csv'
            with open(fname, 'r') as f:
                header = f.readline()
                data = []
                for ns in range(seq_length):
                    new_data = np.array([float(x) for x in f.readline().split(',')[:-1]])
                    sample_rate = new_data[-1]
                    data.append(new_data[:-1])

                data = np.stack(data, axis=0)

            user_batch.append(data)
            user_labels.append(gesture_map[g])

    user_labels = np.array(user_labels)
    user_batch = np.stack(user_batch, axis=0)
    data_dict[u] = (user_batch, user_labels)

In [ ]:
# Extract data
(X, y) = data_dict['david']
X = X[:, :, :, np.newaxis] # add channels axis (1)

#X = X[:, :, 0:5:2] # acceleration data only
X = X[:, :, -4:] # quaternion data only
#X = X[:, :, :6] # acceleration & gyro data only
data_axes = X.shape[2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [ ]:
# Build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(8, (4, data_axes), padding='same', activation='relu', input_shape=(seq_length, data_axes, 1)))
model.add(tf.keras.layers.MaxPool2D((3, data_axes)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Conv2D(16, (4, 1), padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(num_classes))
model.summary()

In [ ]:
# Train model
epochs = 100
batch_size = 8
learning_rate = 1e-3
loss = 'sparse_categorical_crossentropy'
metrics=[tf.keras.metrics.sparse_categorical_accuracy]
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
model.evaluate(X_test, y_test)